# 삭제해야 하는 데이터


---


*   "작성자에 의해 삭제된 댓글입니다."
*   "클린봇이 부적절한 표현을 감지한 댓글입니다."
*   "운영규정 미준수로 인해 삭제된 댓글입니다."
*   Null







# 설치

## Konlpy 설치

In [ ]:
%pip install konlpy

## JDK 설치

In [ ]:
!sudo apt-get install openjdk-11-jdk -y
!sudo apt-get install default-jdk -y

# 데이터 로드


In [ ]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv('/content/drive/MyDrive/공유폴더(딥테크)/comment/파일명') # 파일명 아래에서 복사해서 사용

# GS_comments_2.csv
# KT_comments_2.csv
# SKT_comments_2.csv
# lotte_comments_2.csv

# 데이터 상위 5개 행 확인
display(df.head())

# 텍스트 전처리

In [ ]:
# 불필요한 댓글 목록 정의
unnecessary_comments = [
    "작성자에 의해 삭제된 댓글입니다.",
    "클린봇이 부적절한 표현을 감지한 댓글입니다.",
    "운영규정 미준수로 인해 삭제된 댓글입니다."
]

# 불필요한 댓글을 포함하는 행 삭제
df = df[~df['comment'].isin(unnecessary_comments)]

# 'comment' 열에서 결측치(Null)를 포함하는 행 삭제
df.dropna(subset=['comment'], inplace=True)

# 전처리된 DataFrame의 상위 5개 행 출력하여 결과 확인
display(df.head())

# 형태소 분석


In [ ]:
from konlpy.tag import Okt
import pandas as pd

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 명사, 형용사, 동사, 부사를 추출하는 함수 정의
def extract_pos_tags(text):
    """
    텍스트에서 명사, 형용사, 동사, 부사를 추출합니다.

    Args:
        text (str): 형태소 분석할 텍스트 문자열.

    Returns:
        list: 명사, 형용사, 동사, 부사에 해당하는 형태소 리스트.
    """
    if pd.isna(text):
        return []
    pos_tags = []
    # 형태소 분석 및 품사 태깅
    for word, pos in okt.pos(text, norm=True, stem=True):
        # 명사, 형용사, 동사, 부사만 추출
        if pos in ['Noun', 'Adjective', 'Verb', 'Adverb']:
            pos_tags.append(word)
    return pos_tags

# 'comment' 열에 함수를 적용하여 형태소 분석 결과를 'pos_tags' 열에 저장
df['pos_tags'] = df['comment'].apply(extract_pos_tags)

# 결과 확인
display(df[['comment', 'pos_tags']].head())

## 빈도 분석 및 결과


추출된 형태소들을 모두 하나의 리스트로 합친 후, 각 형태소의 빈도를 계산. 이후 빈도수를 기준으로 내림차순 정렬하여 상위 형태소들을 확인.

In [ ]:
from collections import Counter

# 모든 형태소와 품사를 튜플 형태로 저장
all_pos_tags_with_pos = []
# 품사 매핑 딕셔너리 (영어 -> 한글)
pos_map = {
    'Noun': '명사',
    'Verb': '동사',
    'Adjective': '형용사',
    'Adverb': '부사'
}

for sublist in df['pos_tags']:
    if isinstance(sublist, list): # 리스트 형태인지 확인
        for word in sublist:
            # 단어에 대한 품사 정보를 다시 추출
            pos_info = okt.pos(word, norm=True, stem=True)
            if pos_info:
                 # 첫 번째 결과의 품사를 사용하고 한글로 매핑
                english_pos = pos_info[0][1]
                korean_pos = pos_map.get(english_pos, english_pos) # 매핑 없으면 원래 영어 품사 사용
                all_pos_tags_with_pos.append((word, korean_pos))


# 형태소 및 품사별 빈도 계산
pos_counts_with_pos = Counter(all_pos_tags_with_pos)

# 빈도 분석 결과를 DataFrame으로 생성
# (단어, 품사) 튜플을 인덱스로 사용하여 빈도수를 저장
pos_freq_df = pd.DataFrame.from_dict(pos_counts_with_pos, orient='index', columns=['빈도수'])

# 인덱스에서 단어와 품사를 추출하여 열로 만듦
pos_freq_df.index = pd.MultiIndex.from_tuples(pos_freq_df.index, names=['단어', '품사'])

# DataFrame 재구성 및 열 순서 변경
pos_freq_df = pos_freq_df.reset_index()

# 빈도수를 기준으로 내림차순 정렬
pos_freq_df = pos_freq_df.sort_values(by='빈도수', ascending=False)

# 결과 확인 (상위 20개)
display(pos_freq_df.head(20))

In [ ]:
# pos_freq_df의 행과 열의 개수(크기) 확인
display(pos_freq_df.shape)

In [ ]:
import pandas as pd

# pandas display 옵션 설정: 모든 행을 출력하도록 설정
pd.set_option('display.max_rows', None)

# 전체 DataFrame 출력
display(pos_freq_df)

# display 옵션 초기화 (필요에 따라 주석 해제)
# pd.reset_option('display.max_rows')